# Tricking Stats

In [1]:
import pandas as pd
tricking = pd.read_csv('Tricking.csv')
tricking

,Date,Trick_Name,Attempts,Landed_Count,Surface
0,2025-09-25,Fulls,3,3,Spring Floor
1,2025-09-25,Full Hyperhook,7,7,Spring Floor
2,2025-09-25,Double Full,7,6,Spring Floor
3,2025-09-25,Corks,3,3,Spring Floor
4,2025-09-25,Cheat 1080,2,2,Spring Floor
...,...,...,...,...,...
103,2025-11-20,Cheat 900,3,3,Spring Floor
104,2025-11-20,Cheat 1080,4,4,Spring Floor
105,2025-11-20,Gainer,2,2,Spring Floor
106,2025-11-20,Full Hyperhook,1,1,Spring Floor


In [2]:
tricking['Trick_Name'].unique()

array(['Fulls', 'Full Hyperhook', 'Double Full', 'Corks', 'Cheat 1080',
       'Double B-Twist', 'Cheat 900', 'Gainer', 'Corks (Round)',
       'Round Off Backflip (Multiple Kicks)', 'Double Full Hyperhook',
       'Full Snapu', 'B-Twist Hyperhook', 'B-Twist', 'Double Cork',
       'Round Off Backflip', 'Boxcutter', 'Full Hyperhook (Shuriken)',
       'Triple Full', 'B-Twist (Shuriken)', 'Front Tuck',
       'Back Handspring', 'Wrap Full', 'Wrap Full Hyperhook',
       'Backside 1260', 'Round Off Backflip ', 'Side Swipe'], dtype=object)

### Tricks I've done most often

In [3]:
tricking['Trick_Name'].value_counts().head()

Trick_Name
Full Hyperhook    13
Fulls             11
Corks             11
Double Full        8
Cheat 1080         8
Name: count, dtype: int64

### Show tricks that I didn't land 100% of the time

In [4]:
landed100 = tricking['Attempts'] != tricking['Landed_Count']
tricking[landed100]

,Date,Trick_Name,Attempts,Landed_Count,Surface
2,2025-09-25,Double Full,7,6,Spring Floor
7,2025-09-28,Double B-Twist,3,0,Spring Floor
11,2025-10-05,Corks (Round),7,5,Spring Floor
12,2025-10-05,Cheat 1080,5,3,Spring Floor
22,2025-10-08,Round Off Backflip (Multiple Kicks),3,2,Puzzle Mats
27,2025-10-12,Full Hyperhook,2,1,Spring Floor
29,2025-10-12,Double Full Hyperhook,6,1,Spring Floor
30,2025-10-12,Full Snapu,1,0,Spring Floor
32,2025-10-12,Double B-Twist,1,0,Spring Floor
36,2025-10-17,Double Full Hyperhook,1,0,Spring Floor


np.int64(248)